In [1]:
from cesnet_datazoo.datasets import CESNET_QUIC22
from cesnet_datazoo.config import DatasetConfig, AppSelection, ValidationApproach

dataset = CESNET_QUIC22("~/datasets/CESNET-QUIC22/", size="XS")

common_params = {
    "dataset": dataset,
    "apps_selection": AppSelection.ALL_KNOWN,
    "train_period_name": "W-2022-44",
    "val_approach": ValidationApproach.SPLIT_FROM_TRAIN,
    "train_val_split_fraction": 0.2,
    "use_packet_histograms": True,
}
dataset_config = DatasetConfig(**common_params)
dataset.set_dataset_config_and_initialize(dataset_config)
train_dataframe = dataset.get_train_df(flatten_ppi=True)
val_dataframe = dataset.get_val_df(flatten_ppi=True)
test_dataframe = dataset.get_test_df(flatten_ppi=True)

Loading data from dataloader


100%|██████████| 8162/8162 [00:13<00:00, 591.23it/s] 


Loading data from dataloader


100%|██████████| 192/192 [00:07<00:00, 24.17it/s]


Loading data from dataloader


100%|██████████| 1247/1247 [00:13<00:00, 93.04it/s] 


In [10]:
from sklearn.ensemble import RandomForestClassifier

X_val = val_dataframe.drop(columns="APP").to_numpy()
y_val = val_dataframe["APP"].to_numpy()

clf = RandomForestClassifier(max_depth = 15, n_jobs=-1)

clf.fit(X_val, y_val)


RandomForestClassifier(max_depth=15, n_jobs=-1)

In [15]:
import numpy as np
from sklearn.metrics import accuracy_score

X = train_dataframe.drop(columns="APP").to_numpy()
y = train_dataframe["APP"].to_numpy()

X_test = test_dataframe.drop(columns="APP").to_numpy()[:200_000]
y_test = test_dataframe["APP"].to_numpy()[:200_000]

cols = X.shape[1]
num_of_samples = 100_000

X_chosen = np.ndarray(shape = (num_of_samples, cols))
y_chosen = np.ndarray(shape = (num_of_samples))

# def score(sample):
#     probas = clf.predict_proba(sample.reshape(1, -1))[0]
#     return 1 - np.max(probas)

scores = clf.predict_proba(X)

print("start")


start


In [33]:
num_of_samples = 100000

# for i in range(len(scores)):
#     scores[i] = np.sort(scores[i])[::-1]

X_chosen[:400] = X_val[:400]
y_chosen[:400] = y_val[:400]

t_arr = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
for threshold in t_arr:
    index = 400
    for i in range(num_of_samples):
        if i % 500 == 0:
            clf_test = RandomForestClassifier(max_depth = 15, n_jobs = -1)
            clf_test.fit(X_chosen[:index], y_chosen[:index])

        proba = clf_test.predict_proba(X[i].reshape(1, -1))[0]
        proba = np.sort(proba)[::-1]

        if proba[0] - proba[1] < threshold:
            X_chosen[index] = X[i]
            y_chosen[index] = y[i]
            index += 1

    clf_test = RandomForestClassifier(max_depth = 15, n_jobs = -1)
    clf_test.fit(X_chosen[:index], y_chosen[:index])

    print(index)

    predict_arr = clf_test.predict(X_test)
    print(f"acc of threshold({threshold}) is {accuracy_score(y_test, predict_arr):.4f}")

# clf = RandomForestClassifier(max_depth = 15, n_jobs = -1)
# clf.fit(X[:100000], y[:100000])
# predict_arr = clf.predict(X_test)
# print(f"acc is {accuracy_score(y_test, predict_arr):.4f}")
722


25430
acc of threshold(0.1) is 0.7692
37004
acc of threshold(0.2) is 0.7728
46134
acc of threshold(0.3) is 0.7732
54067
acc of threshold(0.4) is 0.7654
60981
acc of threshold(0.5) is 0.7575
67179
acc of threshold(0.6) is 0.7510
73526
acc of threshold(0.7) is 0.7474
79805
acc of threshold(0.8) is 0.7470
86318
acc of threshold(0.9) is 0.7436
98254
acc of threshold(1) is 0.7373


722